In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col,split,lit,broadcast


# Create a SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

# Disable broadcast joins
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

#for SPJ to be enabled it should have minimal of these 2 configs set:
# Setting SPJ related configs
spark.conf.set('spark.sql.sources.v2.bucketing.enabled','true') 
spark.conf.set('spark.sql.sources.v2.bucketing.pushPartValues.enabled','true')
spark.conf.set('spark.sql.iceberg.planning.preserve-data-grouping','true')
spark.conf.set('spark.sql.requireAllClusterKeysForCoPartition','false')
spark.conf.set('spark.sql.sources.v2.bucketing.partiallyClusteredDistribution.enabled','true')



# Read the match_details CSV file
match_details = spark.read.option("header", "true").csv("/home/iceberg/data/match_details.csv")

# Read the match CSV file
matches = spark.read.option("header", "true").csv("/home/iceberg/data/matches.csv")

# Read the medals_matches_players CSV file
medals_matches_players = spark.read.option("header", "true").csv("/home/iceberg/data/medals_matches_players.csv")

# Read the medals CSV file
medals = spark.read.option("header", "true").csv("/home/iceberg/data/medals.csv")


# Read the maps CSV file
maps = spark.read.option("header", "true").csv("/home/iceberg/data/maps.csv")




24/12/15 00:15:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
#Creating the Bootcamp Database

%%sql
CREATE DATABASE IF NOT EXISTS bootcamp


++
||
++
++

In [3]:
## Dropping the match_details if exists schema 

%%sql
DROP TABLE IF EXISTS bootcamp.match_details

++
||
++
++

In [4]:
#Creating the match_details schema 

%%sql
CREATE TABLE IF NOT EXISTS bootcamp.match_details (
     match_id STRING,
     player_gamertag STRING,
     player_total_kills INTEGER,
     player_total_deaths INTEGER
)
USING iceberg
PARTITIONED BY (bucket(16, match_id))


++
||
++
++

In [5]:
#Writing the match_details data into the match_details spark table 

match_details.select("match_id", "player_gamertag","player_total_kills", "player_total_deaths") \
.write.mode("overwrite") \
.bucketBy(16, "match_id").saveAsTable("bootcamp.match_details") 
  

print(f"Match details records count : {match_details.count()}")

Match details records count : 151761


In [6]:
## Dropping the matches schema 

%%sql
DROP TABLE IF EXISTS bootcamp.matches

++
||
++
++

In [42]:
#Creating the medals schema 

spark.sql("""
CREATE TABLE IF NOT EXISTS bootcamp.matches (
        match_id STRING,
        mapid STRING,
        is_team_game STRING,
        playlist_id STRING,
        completion_date TIMESTAMP
  )
   USING iceberg
   PARTITIONED BY (bucket(16, match_id))
   """)

DataFrame[]

In [8]:
#Writing the matches data into the matches spark table 

matches.select("match_id", "mapid", "is_team_game", "is_match_over", "playlist_id", "completion_date") \
.write.mode("overwrite") \
.bucketBy(16, "match_id").saveAsTable("bootcamp.matches") 
  

print(f"Matches records count : {matches.count()}")

Matches records count : 24025


In [9]:
## Dropping the medals_matches_players schema 

%%sql
DROP TABLE IF EXISTS bootcamp.medals_matches_players

++
||
++
++

In [10]:
#Creating the medals_matches_players schema 

%%sql
CREATE TABLE IF NOT EXISTS bootcamp.medals_matches_players (
        match_id STRING,
        player_gamertag STRING,
        medal_id STRING,
        count INTEGER
  )
   USING iceberg
   PARTITIONED BY (bucket(16, match_id));


++
||
++
++

In [11]:
# Writing into the  medals_matches_players Schema
medals_matches_players.select("match_id", "player_gamertag","medal_id", "count",) \
     .write.mode("overwrite") \
     .bucketBy(16, "match_id").saveAsTable("bootcamp.medals_matches_players") 

print(f"Medals Match Player details records count : {medals_matches_players.count()}")

Medals Match Player details records count : 755229


In [12]:
# Save the Medals DataFrame as a table
medals.write.mode("overwrite").saveAsTable("bootcamp.medals")
# medals.createOrReplaceTempView("bootcamp.medals")

# Save the Maps DataFrame as a table
maps.write.mode("overwrite").saveAsTable("bootcamp.maps")
# maps.createOrReplaceTempView("bootcamp.maps")


In [14]:
#Bucket join and broadcast join to get merged aggregate data frame to answer questions
spark.table("bootcamp.matches").alias("mtc") \
.join(spark.table("bootcamp.match_details").alias("mth_det"), col("mtc.match_id") == col("mth_det.match_id"), "left") \
.join(spark.table("bootcamp.medals_matches_players").alias("medals_mth_players"), ((col("mtc.match_id") == col("medals_mth_players.match_id")) &
(col("mth_det.player_gamertag") == col("medals_mth_players.player_gamertag"))), "left") \
.join(broadcast(spark.table("bootcamp.medals")).alias("medls"), col("medals_mth_players.medal_id") == col("medls.medal_id"), "inner") \
.join(broadcast(spark.table("bootcamp.maps")).alias("mps"), col("mtc.mapid") == col("mps.mapid"), "inner") \
.select(col("mtc.match_id"),col("mtc.mapid"),col("mps.name").alias("map_name"),col("medals_mth_players.medal_id"),
col("medls.name").alias("medal_name"),col("mtc.playlist_id"),col("mtc.completion_date"), \
col("mth_det.player_gamertag"), col("classification"), col("mth_det.player_total_kills"),col("medals_mth_players.count")).write.mode("overwrite").saveAsTable("bootcamp.final_data")

In [15]:
# viewing the final aggregated merged data

%%sql 
select * from bootcamp.final_data


match_id,mapid,map_name,medal_id,medal_name,playlist_id,completion_date,player_gamertag,classification,player_total_kills,count
0000e3cf-727c-491a-9de8-43fe6ea611cc,ce89a40f-f206-11e4-b83f-24be05e24f7e,None,2078758684,Double Kill,d0766624-dbd7-4536-ba39-2d890a6143a9,2016-05-09 00:00:00.000000,Draper,MultiKill,23,2
0000e3cf-727c-491a-9de8-43fe6ea611cc,ce89a40f-f206-11e4-b83f-24be05e24f7e,None,2430242797,Killing Spree,d0766624-dbd7-4536-ba39-2d890a6143a9,2016-05-09 00:00:00.000000,Draper,KillingSpree,23,1
0000e3cf-727c-491a-9de8-43fe6ea611cc,ce89a40f-f206-11e4-b83f-24be05e24f7e,None,250435527,Brawler,d0766624-dbd7-4536-ba39-2d890a6143a9,2016-05-09 00:00:00.000000,Draper,Style,23,1
0000e3cf-727c-491a-9de8-43fe6ea611cc,ce89a40f-f206-11e4-b83f-24be05e24f7e,None,3261908037,Headshot,d0766624-dbd7-4536-ba39-2d890a6143a9,2016-05-09 00:00:00.000000,Draper,WeaponProficiency,23,4
0000e3cf-727c-491a-9de8-43fe6ea611cc,ce89a40f-f206-11e4-b83f-24be05e24f7e,None,1986137636,Snapshot,d0766624-dbd7-4536-ba39-2d890a6143a9,2016-05-09 00:00:00.000000,Draper,WeaponProficiency,23,1
0000e3cf-727c-491a-9de8-43fe6ea611cc,ce89a40f-f206-11e4-b83f-24be05e24f7e,None,2287626681,Melee Kill,d0766624-dbd7-4536-ba39-2d890a6143a9,2016-05-09 00:00:00.000000,Draper,Style,23,2
0000e3cf-727c-491a-9de8-43fe6ea611cc,ce89a40f-f206-11e4-b83f-24be05e24f7e,None,848240062,Sniper Headshot,d0766624-dbd7-4536-ba39-2d890a6143a9,2016-05-09 00:00:00.000000,Draper,WeaponProficiency,23,1
0000e3cf-727c-491a-9de8-43fe6ea611cc,ce89a40f-f206-11e4-b83f-24be05e24f7e,None,2782465081,Reversal,d0766624-dbd7-4536-ba39-2d890a6143a9,2016-05-09 00:00:00.000000,Mike117Rune,Style,19,2
0000e3cf-727c-491a-9de8-43fe6ea611cc,ce89a40f-f206-11e4-b83f-24be05e24f7e,None,3261908037,Headshot,d0766624-dbd7-4536-ba39-2d890a6143a9,2016-05-09 00:00:00.000000,Mike117Rune,WeaponProficiency,19,3
0000e3cf-727c-491a-9de8-43fe6ea611cc,ce89a40f-f206-11e4-b83f-24be05e24f7e,None,250435527,Brawler,d0766624-dbd7-4536-ba39-2d890a6143a9,2016-05-09 00:00:00.000000,Mike117Rune,Style,19,1


In [16]:
# Question: Which player averages the most kills per game?
# Query- Answer

%%sql

SELECT player_gamertag, match_id, Avg(player_total_kills) AS kills
FROM bootcamp.final_data
GROUP BY player_gamertag, match_id
ORDER BY kills DESC
LIMIT(1)

player_gamertag,match_id,kills
gimpinator14,acf0e47e-20ac-4b12-b292-591d4b3a3df9,109.0


In [17]:
# Question: Which playlist gets played the most?
# Query- Answer

%%sql

SELECT playlist_id, COUNT(Distinct match_id) AS count
FROM bootcamp.final_data
GROUP BY playlist_id
ORDER BY count DESC
LIMIT(1)

playlist_id,count
f72e0ef0-7c4a-4307-af78-8e38dac3fdba,7640


In [41]:
# Question: Which map gets played the most?
# Query- Answer

%%sql
    
SELECT map_name,count(distinct match_id) AS count
FROM bootcamp.final_data
GROUP BY map_name
ORDER BY count DESC
LIMIT(1)

map_name,count
Breakout Arena,7032


In [18]:
# Question: Which map do players get the most Killing Spree medals on?
# Query- Answer

%%sql
    
SELECT map_name,Sum(Count) AS count
FROM bootcamp.final_data
WHERE classification = "KillingSpree"
GROUP BY map_name
ORDER BY count DESC
LIMIT(1)

map_name,count
Breakout Arena,6919.0


In [19]:
#reading the final data back into dataframe
final_df = spark.read.table("bootcamp.final_data")

In [34]:
##Trying different partition columns and number to see which has the smallest data size

#local partition sort
sorted = final_df.repartition(5, col("match_id")) \
                .sortWithinPartitions(col("playlist_id"),col("map_name"),col("match_id"))


#global sort 
unsorted = final_df.repartition(5, col("match_id")) \
            .sort(col("playlist_id"),col("map_name"),col("match_id"))


# sorted.show(5)
# unsorted.show(5)
sorted.explain()
unsorted.explain()

sorted.write.mode("overwrite").saveAsTable("bootcamp.finaldf_sorted")
unsorted.write.mode("overwrite").saveAsTable("bootcamp.finaldf_unsorted")

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [playlist_id#842 ASC NULLS FIRST, map_name#839 ASC NULLS FIRST, match_id#837 ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(match_id#837, 5), REPARTITION_BY_NUM, [plan_id=2430]
      +- BatchScan demo.bootcamp.final_data[match_id#837, mapid#838, map_name#839, medal_id#840, medal_name#841, playlist_id#842, completion_date#843, player_gamertag#844, classification#845, player_total_kills#846, count#847] demo.bootcamp.final_data (branch=null) [filters=, groupedBy=] RuntimeFilters: []


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [playlist_id#842 ASC NULLS FIRST, map_name#839 ASC NULLS FIRST, match_id#837 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(playlist_id#842 ASC NULLS FIRST, map_name#839 ASC NULLS FIRST, match_id#837 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=2445]
      +- Exchange hashpartitioning(match_id#837, 5), REPARTITION_BY_NUM, [plan_id=2442]
         +- Ba

In [35]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.finaldf_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.finaldf_unsorted.files


size,num_files,sorted
3922394,5,sorted
3798881,8,unsorted
